In [3]:
#This is a Ganymede Auroral Zone only codebuilt from the Prime Ganymede code


""" SET UP
- import necessary libraries
"""


#Import libraries
import pandas as pd
import numpy as np
import scipy
from scipy import integrate
from scipy import interpolate
import glob
import re
import math
import os
import plotly.express as px
import plotly.graph_objects as go

#change to working directory
os.chdir('/Users/hwaite/Desktop/JUNO 2016 2022/JADEganymede/GanymedCodeWithInputFiles')

#  First version began in Fall 2021. Includes ion production from electron impact on H2, O2, and H2O
# as well as dissociative excitation of OI1305 and OI1356. Uses JADE data set from 165200 to 165899
# UTC. Cross sections from literature as noted below. Also uses Ben Teolis's atmospheric model with
# scaling inferred from Roth et al HST paper and enhanced O2 to match UVS PJ34 flyby.
#
# Updates listed in Early February of 2022: 1) Added JADE magnetopuase electron energy flux for aurora,
# 2) Added additional energy loss processes in O2 and H2 - Dissociation of O2 to fill energy loss 
# gap in the range of 40 to 100 eV, H2 & O2 vibrational excitation, O2 total scattering, 3) added
# in Maxwellians constrained by WAVES density and matched to JADE extrtapolated to 13 eV,
# 4) Added in photoionization rates from Huebner compilation for H2, O2, and H2O,5) Added in
# estimate of H2+ density using ion production and transport to use in determining the production
# of H3+, which also required adding in the JUNO occultation profile to get the ionospheric
# loss rates correct for H2+.
#
# This version includes the energy deposition calculation combined with the electron
# temperature calculation portion of the code. It aslso provides a MAxwelleian check
# on the Eviatar emission intensity using a 100 eV Maxwellian.
# Now converted to auroral zone only and near final I hope.


#Read in Rob Ebert's magnetopause spectrum at 170100
aselecMP_header = ["EMP", "PMP", "UTC"]
dfelecMP = pd.read_csv('REganMPavg.csv',names=aselecMP_header,skiprows=1)
dfelecMP = dfelecMP.apply(pd.to_numeric)

#Scale electron fluxes to per eV in the downward precipitating loss cone of pi steradians
# Electron flux scaling factor
cortfac = 3.14159e-3

#Define interpolation scheme to determine the electron flux at an arbitrary energy
#Use linear interpolation in a log flux/log energy parameter space

#Define fit range and interpolate electron flux to energy e and change to per ev per 1.84 ster

def eflxforcs(e,xl,yl,Neval,Teval):
    if e<xl[0]:
        yint = eflxblw13(e,Neval,Teval)
        print("called eflxblw13",yint)
        return yint
    else:
        x = np.log10(xl)
        y = np.log10(yl)
        enew = np.log10(e)
        f = interpolate.interp1d(x, y, kind = 'linear',fill_value ="extrapolate",assume_sorted=False)
        base = f(enew)
        yint = 10**base
        if yint<1e-3:
            print(yint)
        return yint

#Read in Electron Impact Ionization Cross sections

#Cross sections for H2, O2, and H2O in cm^2
asEIH2_header = ["E", "H2+", "H+"]
dfCSH2 = pd.read_csv('EI_H2_CS.csv',names=asEIH2_header,skiprows=1)
dfCSH2 = dfCSH2.apply(pd.to_numeric)
dfCSH2["H2+"] = dfCSH2["H2+"]*1.e-17
dfCSH2["H+"] = dfCSH2["H+"]*1.e-18

#cross sections for O2 in cm^2
asEIO2_header = ["E", "O2+", "O+", "O++"]
dfCSO2 = pd.read_csv('EI_O2_CS.csv',names=asEIO2_header,skiprows=1)
dfCSO2 = dfCSO2.apply(pd.to_numeric)
dfCSO2["O2+"] = dfCSO2["O2+"]*1.e-16
dfCSO2["O+"] = dfCSO2["O+"]*1.e-17
dfCSO2["O++"] = dfCSO2["O++"]*1.e-18

#Cross sections for H2O in cm^2
#The H2O+ includes OH+ and O+ as well as H2O+
asEIH2O_header = ["E", "H2O+", "O++", "H2+", "H+"]
dfCSH2O = pd.read_csv('EI_H2O_CS.csv',names=asEIH2O_header,skiprows=1)
dfCSH2O = dfCSH2O.apply(pd.to_numeric)
dfCSH2O["H2O+"] = dfCSH2O["H2O+"]*1.e-16
dfCSH2O["O++"] = dfCSH2O["O++"]*1.e-19
dfCSH2O["H2+"] = dfCSH2O["H2+"]*1.e-19
dfCSH2O["H+"] = dfCSH2O["H+"]*1.e-17

#Read in Dissociative Excitation Cross sections

#Electron impact Dissociative Excitation Cross sections for O and H2O in cm^2

#cross sections for O2 in cm^2
#O2 DisEx comes from Kanik et al.JGR, 108(E11),5126, doi:10.1029/2000JE001423,2003

#e- on O2 --> 1304
#Uses Table2 to 600 eV then Aartes and De Heers [1971] column scaled to go to higher energies
# and finally extends slope using Aartes from 1000 to 5000 eV to reach end of file energy
asDEO21304_header = ["EO21304", "CS1304Mb"]
dfCSdeO21304 = pd.read_csv('DisExeO21304.csv',names=asDEO21304_header,skiprows=1)
dfCSdeO21304 = dfCSdeO21304.apply(pd.to_numeric)
dfCSdeO21304["CS1304Mb"] = dfCSdeO21304["CS1304Mb"]*1.e-18
#e- on O2 --> 1356
#Uses Table3 to 600 eV then follows shape function of 1304 above to end of eenergy range of file
asDEO21356_header = ["EO21356", "CS1356Mb"]
dfCSdeO21356 = pd.read_csv('DisExeO21356.csv',names=asDEO21356_header,skiprows=1)
dfCSdeO21356 = dfCSdeO21356.apply(pd.to_numeric)
dfCSdeO21356["CS1356Mb"] = dfCSdeO21356["CS1356Mb"]*1.e-18

#print(dfCSdeO21304,dfCSdeO21356)

#Cross sections for H2O in cm^2
#H2O DisEx comes from Makarov et al., JGR, 109, A09303, doi:10.1029/2002JA009353, 2004
#e- on H2O --> 1304
#Uses Table 3 and extends using O2 shape function as indicated above to higher energy
asDEH2O1304_header = ["EH2O1304", "CS1304Mb$10"]
dfCSdeH2O1304 = pd.read_csv('DisExeH2O1304.csv',names=asDEH2O1304_header,skiprows=1)
dfCSdeH2O1304 = dfCSdeH2O1304.apply(pd.to_numeric)
dfCSdeH2O1304["CS1304Mb$10"] = dfCSdeH2O1304["CS1304Mb$10"]*5.385e-19
#e- on O2 --> 1356
#Uses Table 1 to get ratio of total 1304 to 1356 and folows the 1304 shape function
asDEH2O1356_header = ["EH2O1356", "CS1356Mb$10"]
dfCSdeH2O1356 = pd.read_csv('DisExeH2O1356.csv',names=asDEH2O1356_header,skiprows=1)
dfCSdeH2O1356 = dfCSdeH2O1356.apply(pd.to_numeric)
dfCSdeH2O1356["CS1356Mb$10"] = dfCSdeH2O1356["CS1356Mb$10"]*1.e-20

#Read in cross sections for other e on O2 processes to use to check production or
#energy balance and loss
asQTeo2_header = ["EO2QT", "CSQTeO2"]
dfCSQTo2 = pd.read_csv('QTscatO2.csv',names=asQTeo2_header,skiprows=1)
dfCSQTo2 = dfCSQTo2.apply(pd.to_numeric)
asvibeo2_header = ["EO2vib", "CSvibeO2"]
dfCSvibo2 = pd.read_csv('O2 Vib.csv',names=asvibeo2_header,skiprows=1)
dfCSvibo2 = dfCSvibo2.apply(pd.to_numeric)
asB3sigeo2_header = ["EO2B3sig", "CSB3sigeO2"]
dfCSB3sigo2 = pd.read_csv('BtripletSigmaDissociation.csv',names=asB3sigeo2_header,skiprows=1)
dfCSB3sigo2 = dfCSB3sigo2.apply(pd.to_numeric)
asvibh2_header = ["EH2vib", "CSvibeH2"]
dfCSvibh2 = pd.read_csv('eCSH2vibSchulzWestinghouse.csv',names=asvibh2_header,skiprows=1)
dfCSvibh2 = dfCSvibh2.apply(pd.to_numeric)

def eflxblw13(e,Neval,Teval):
    rkbev = 8.6171e-5
    rkberg = 1.3805e-16
    elecmass = 9.1091e-28
    pi = 3.14159
    elecmeanvel = math.sqrt((8.0*rkberg*Teval)/(elecmass*pi))
    x = e/(rkbev*Teval)
    eflux = (Neval*elecmeanvel/4.0)*x*np.exp(-x)
    return eflux


#Read in atmospheric densities as a afunction of altitude in meters
asAtm_header = ["alt", "H2On", "O2n", "H2n", "H2Om", "O2m", "H2m"]
dfAtm = pd.read_csv('AVatmosphere.csv',names=asAtm_header,skiprows=3)
dfAtm = dfAtm.apply(pd.to_numeric)

#Calculate the mean free path in the atmosphere the surface to 5000 km.according to the altitude profile given in the Teolis
#spreadsheet.
#The formula for mean free path is given by λ= 1/(sqrt(2)*pi*d^2*n)  
#n is the astmospheric density, pi*d^2= The effective cross section of
#the gas particles, which we assume is 4.^-19 m^2 for O2
#The scale factors for night and day are shown in green based on Roth et al HST observations and auroal emission constraints

import numpy as np
altgrd = np.array(dfAtm['alt'])
altgrd = np.array(altgrd*1.0e2)
nh2oden = np.array(dfAtm['H2On'])
nh2oden = np.array(nh2oden/1.0e+6)
no2den = np.array(dfAtm['O2n'])
no2den = np.array(no2den/1.0e+6)
nh2den = np.array(dfAtm['H2n'])
nh2den = np.array(nh2den/1.0e+6)

mh2oden = np.array(dfAtm['H2Om'])
mh2oden = np.array(mh2oden/1.0e+6)
mo2den = np.array(dfAtm['O2m'])
mo2den = np.array(mo2den/1.0e+6)
mh2den = np.array(dfAtm['H2m'])
mh2den = np.array(mh2den/1.0e+6)

# O2 noise fix
for i in range(len(altgrd)):
    if no2den[i]<1.e-1:
        no2den[i]=1.e-1
    if mo2den[i]<1.e-1:
        mo2den[i]=1.e-1
        
ntden = nh2oden+no2den+nh2den
mtden = mh2oden+mo2den+mh2den
nso2mfp = 5.20e-15*no2den*no2den/ntden
nsh2omfp = 1.66e-15*nh2oden*nh2oden/ntden
nsh2mfp = 2.68e-15*nh2den*nh2den/ntden
no2h2omfp = 1./4.*(5.20e-15**0.5+1.66e-15**0.5)**2*no2den*nh2oden/ntden
no2h2mfp = 1./4.*(5.20e-15**0.5+2.68e-15**0.5)**2*no2den*nh2den/ntden
nh2oh2mfp = 1./4.*(1.66e-15**0.5+2.68e-15**0.5)**2*nh2oden*nh2den/ntden
nmfpm = (1./(2**0.5*(nso2mfp+nsh2omfp+nsh2mfp+no2h2omfp+no2h2mfp+nh2oh2mfp)))
mso2mfp = 5.20e-15*mo2den*mo2den/mtden
msh2omfp = 1.66e-15*mh2oden*mh2oden/mtden
msh2mfp = 2.68e-15*mh2den*mh2den/mtden
mo2h2omfp = 1./4.*(5.20e-15**0.5+1.66e-15**0.5)**2*mo2den*mh2oden/mtden
mo2h2mfp = 1./4.*(5.20e-15**0.5+2.68e-15**0.5)**2*mo2den*mh2den/mtden
mh2oh2mfp = 1./4.*(1.66e-15**0.5+2.68e-15**0.5)**2*mh2oden*mh2den/mtden
mmfpm = (1./(2**0.5*(mso2mfp+msh2omfp+msh2mfp+mo2h2omfp+mo2h2mfp+mh2oh2mfp)))
dfMFP = pd.DataFrame(data={'ALTITUDE': altgrd, 'noon mean free path m': nmfpm, 'midnight mean free path m': mmfpm})
#Plot mean free path in km versus altitude in kilometers                     
figmfp = px.scatter(x=nmfpm/1.0e5,y=altgrd/1.0e5,log_x=1,title="Mean Free Path (km)",labels={"y":  "Altitude(km)", "x": "Mean Free Path(km)"})
figmfp.add_scatter(x=nmfpm/1.0e5,y=altgrd/1.0e5,name= 'Noon')
figmfp.add_scatter(x=mmfpm/1.0e5,y=altgrd/1.0e5,name= 'Midnight')
figmfp.update_xaxes(exponentformat='E', title='Mean Free Path (km)')
figmfp.show()
#Plot for displaying the atmospheric density vs altitude
#noon
figatmn = px.scatter(x=no2den, y=altgrd/1.0e5, log_x=1, title='Atmospheric densities (cm^-3)',labels={"y":  "Altitude(km)", "x": "Density(cm^-3)"})
figatmn.add_scatter(x=no2den, y=altgrd/1.0e5, mode='markers',name='Day O2')
figatmn.add_scatter(x=nh2oden, y=altgrd/1.0e5, mode='markers', name='Day H2O')
figatmn.add_scatter(x=nh2den, y=altgrd/1.0e5,  mode='markers',name='Day H2')
figatmn.update_xaxes(exponentformat="E")
figatmn.show()
# Determine atmospheric column densities on dayside
colnh2oden = integrate.simpson(nh2oden,altgrd,even='avg')
colno2den = integrate.simpson(no2den,altgrd,even='avg')
colnh2den = integrate.simpson(nh2den,altgrd,even='avg')
print("NOON Column (cm^-2)", "H2O =", "{:.2e}".format(colnh2oden), "O2", "{:.2e}".format(colno2den),"H2", "{:.2e}".format(colnh2den))
#midnight
figatmm = px.scatter(x=mo2den, y=altgrd/1.0e5, log_x=1, title='Atmospheric densities (cm^-3)',labels={"y":  "Altitude(km)", "x": "Density(cm^-3)"})
figatmm.add_scatter(x=mo2den, y=altgrd/1.0e5, mode='markers',name='Night O2')
figatmm.add_scatter(x=mh2oden, y=altgrd/1.0e5, mode='markers', name='Night H2O')
figatmm.add_scatter(x=mh2den, y=altgrd/1.0e5,  mode='markers',name='Night H2')
figatmm.update_xaxes(exponentformat="E")
figatmm.show()
# Deternmmine column densities on the nightside
colmh2oden = integrate.simpson(mh2oden,altgrd,even='avg')
colmo2den = integrate.simpson(mo2den,altgrd,even='avg')
colmh2den = integrate.simpson(mh2den,altgrd,even='avg')

print("MIDNIGHT Column (cm^-2)", "H2O =", "{:.2e}".format(colmh2oden), "O2", "{:.2e}".format(colmo2den),"H2", "{:.2e}".format(colmh2den))

# estimate measured electron density profiles from Juno RSS
#Function to determine the sample ionospheric density in cm-3 from Dustin ingress n0=2000cm-3 T=490 for O2+
#Note that the altitutde grid is in meters; ingress uses eflg=0 and ingress eflg=1
def electrondensity(eflg,z):
    if eflg ==0:
# This is the ingress ionsophere
        if z<1.5e7:
            edenval = 2.25e3*np.exp(-z/3.3e7)
        else:
            if z >= 3.5e7:
                edenval = 1.0e3*np.exp(-z/1.23e8)
            else:
                edenval = 1.7e3*np.exp(-z/3.8e7)
    elif eflg==1:            
# This is for the egress ionosphere
        edenval = 1.2e3*np.exp(-z/1.64e7)
    if edenval < 1.:
            edenval = 1.
    return edenval
# Generate an electron density altitude profile
def elecaltprofile(eflg,altgrd):
    index = len(altgrd)-1
    elecprofile = np.zeros_like(altgrd)
    for i in range(index):
        z = altgrd[i]
        elecprofile[i] =  electrondensity (eflg,z)
    return elecprofile

#Cross check the expected photolysis production using Huebner reference
photh2h2p = np.array(nh2den*5.41e-8/27.04)
photh2hp = np.array(nh2den*9.52e-9/27.04)
photo2o2p = np.array(no2den*4.67e-7/27.04)
photo2op = np.array(no2den*1.1e-7/27.04)
photh2ohp = np.array(nh2oden*1.31e-8/27.04)
photh2oh2op = np.array(nh2oden*3.31e-7/27.04)
photh2oop = np.array(nh2oden*5.85e-9/27.04)
photh2oohp = np.array(nh2oden*5.54e-8/27.04)

#Plot photolysis production rates
figphot = px.scatter(x=photo2o2p, y=altgrd/1.0e5, log_x=1, title='Photolysis production rates (cm^-3 s^-1)',labels={"y":  "Altitude (km)", "x": "Photolysis Production (cm^-3 s^-1)"})
figphot.add_scatter(x=photo2o2p, y=altgrd/1.0e5, mode='markers',name='O2+')
figphot.add_scatter(x=photh2h2p, y=altgrd/1.0e5, mode='markers', name='H2+')
figphot.add_scatter(x=(photh2hp+photh2ohp), y=altgrd/1.0e5,  mode='markers',name='H+')
figphot.add_scatter(x=(photo2op+photh2oh2op+photh2oop+photh2oohp), y=altgrd/1.0e5,  mode='markers',name='WG+')
figphot.update_xaxes(exponentformat="E")
figphot.show()

#column integrate
cphoth2h2p = integrate.simpson(photh2h2p,altgrd,even='avg')
cphoth2hp = integrate.simpson(photh2hp,altgrd,even='avg')
cphoto2o2p = integrate.simpson(photo2o2p,altgrd,even='avg')
cphoto2op = integrate.simpson(photo2op,altgrd,even='avg')
cphoth2ohp = integrate.simpson(photh2ohp,altgrd,even='avg')
cphoth2oh2op = integrate.simpson(photh2oh2op,altgrd,even='avg')
cphoth2oop = integrate.simpson(photh2oop,altgrd,even='avg')
cphoth2oohp = integrate.simpson(photh2oohp,altgrd,even='avg')
#mass 2
print('Photolysis producing H2+',"{:.2e}".format(cphoth2h2p))
#mass 1
cphothp = cphoth2hp + cphoth2ohp
print('Photolysis producing H+',"{:.2e}".format(cphothp))
#mass ~16
cphotm16 = cphoto2op + cphoth2oh2op + cphoth2oop + cphoth2oohp
print('Photolysis producing m16+',"{:.2e}".format(cphotm16))
# mass 32
print('Photolysis producing O2+',"{:.2e}".format(cphoto2o2p))
                    
ProdO2Ndf = pd.DataFrame()
ProdH2Ndf = pd.DataFrame()
ProdH2ONdf = pd.DataFrame()
ProdO2Mdf = pd.DataFrame()
ProdH2Mdf = pd.DataFrame()
ProdH2OMdf = pd.DataFrame()
ProdERPdf = pd.DataFrame()
CProdO2Ndf = pd.DataFrame()
CProdH2Ndf = pd.DataFrame()
CProdH2ONdf = pd.DataFrame()
CProdO2Mdf = pd.DataFrame()
CProdH2Mdf = pd.DataFrame()
CProdH2OMdf = pd.DataFrame()


dfenergy = np.array(dfelecMP.EMP,dtype=float)
dfelecflux = np.array(dfelecMP.PMP*cortfac,dtype=float) 
Teval = 2.5e4
Neval = 1.
xpass = np.array(dfenergy)
ypass = np.array(dfelecflux)
   
                  
#Calculate energy dependent production rates
#These do not depend on altitude or day vs night since we assume optically thin electron influx conditions
                  
#ionization of O2
eno2 = np.array(dfCSO2["E"],dtype=float)
cso2o2p = np.array(dfCSO2["O2+"],dtype=float)
cso2op = np.array(dfCSO2["O+"],dtype=float)
cso2opp = np.array(dfCSO2["O++"],dtype=float)

#ionization of O2
eno2 = np.array(dfCSO2["E"],dtype=float)
cso2o2p = np.array(dfCSO2["O2+"],dtype=float)
cso2op = np.array(dfCSO2["O+"],dtype=float)
cso2opp = np.array(dfCSO2["O++"],dtype=float)
#    print(eno2,cso2o2p,cso2op)
#log interpolate the electron energy flux at energies eno2 and load into array elecflux
nepts = len(eno2)
csefo2o2p = np.zeros_like(cso2o2p)
csefo2op = np.zeros_like(cso2op)
csefo2opp = np.zeros_like(cso2opp)
se0fac = 0.0
cssum0o2 = 0.0
se1fac = 0.0
cssum1o2 = 0.0
for j in range(nepts):
    e = eno2[j]
    ef = eflxforcs(e,xpass,ypass,Neval,Teval)
    csefo2o2p[j] = ef*cso2o2p[j]
    csefo2op[j] = ef*cso2op[j]
    csefo2opp[j] = ef*cso2opp[j]
    
# include estimated effect of secondary electrons
    if (e>1746.):
        se1fac=0.45
    else:
        if (e<1746.) and (e>=478.):
            se1fac = 0.35
        else:
            if (e<478.) and (e>=85.):
                se1fac = 0.25
    if (e<85.):
        se0fac = 0.091
    cssum0o2 = cssum0o2 + (csefo2o2p[j] + csefo2op[j] + csefo2opp[j])*se0fac
    cssum1o2 = cssum1o2 + (csefo2o2p[j] + csefo2op[j] + csefo2opp[j])*se1fac
    
#add in secondaries for sum0 to energy bin near 13 ev and for sum1 for energy bin near 36 eV
    
#secondary electrons that map to 13 eV
e = eno2[0]
ef = eflxforcs(e,xpass,ypass,Neval,Teval)
#store primary electron ionization p;roduction in lowest energy bin
pelo2o2p13 = csefo2o2p[0]
pelo2op13 = csefo2op[0]
pelo2opp13 = csefo2opp[0]
#store secondary electron ionization production in lowest energy bin
seplo2o2p13 = ef*cso2o2p[0]*cssum0o2
seplo2op13 = ef*cso2op[0]*cssum0o2
seplo2opp13 = ef*cso2opp[0]*cssum0o2
#store energy loss in lowest energy bin
elo2o2p13 = seplo2o2p13*13.
elo2op13 = seplo2op13*13.
elo2popp13 = seplo2opp13*13.
#add secondary production with primary prouction for future calculation with altitude
irefo2o2p13 = seplo2o2p13/csefo2o2p[0]
csefo2o2p[0] = csefo2o2p[0] + seplo2o2p13
irefo2op13 = seplo2op13/csefo2op[0]
csefo2op[0] = csefo2op[0] + seplo2op13
irefo2op13 = seplo2opp13/csefo2opp[0]
csefo2opp[0] = csefo2opp[0] + seplo2opp13
#repeat for higher energy secondary electrons produced
e = eno2[6]
ef = eflxforcs(e,xpass,ypass,Neval,Teval)
seplo2o2p35 = ef*cso2o2p[6]*cssum1o2
seplo2op35 = ef*cso2op[6]*cssum1o2
seplo2opp35 = ef*cso2opp[6]*cssum1o2
elo2o2p35 = seplo2o2p35*35.
elo2op35 = seplo2op35*35.
elo2popp35 = seplo2opp35*35.
irefo2o2p35 = seplo2o2p35/csefo2o2p[6]
csefo2o2p[6] = csefo2o2p[6] + seplo2o2p35
irefo2op35 = seplo2op35/csefo2op[6]
csefo2op[6] = csefo2op[6] + seplo2op35
irefo2op35 = seplo2opp35/csefo2opp[6]
csefo2opp[6] = csefo2opp[6] + seplo2opp35


#integrate over  energy
pfo2o2p = integrate.simpson(csefo2o2p,eno2,even='avg')
pfo2op = integrate.simpson(csefo2op,eno2,even='avg')
pfo2opp =integrate.simpson(csefo2opp,eno2,even='avg')

#ionization of H2
enh2 = np.array(dfCSH2["E"],dtype=float)
csh2h2p = np.array(dfCSH2["H2+"],dtype=float)
csh2hp = np.array(dfCSH2["H+"],dtype=float)
nepts = len(enh2)
csefh2h2p = np.zeros_like(csh2h2p)
csefh2hp = np.zeros_like(csh2hp)
csefo2opp = np.zeros_like(cso2opp)
se0fac = 0.
cssum0h2 = 0.
se1fac =0.
cssum1h2 = 0.
for j in range(nepts):
    e = enh2[j]
    ef = eflxforcs(e,xpass,ypass,Neval,Teval)
    csefh2h2p[j] = ef*csh2h2p[j]
    csefh2hp[j] = ef*csh2hp[j]
    
# include estimated effect of secondary electrons
    if (e>1746.):
        se1fac=0.45
    else:
        if (e<1746.) and (e>=478.):
            se1fac = 0.35
        else:
            if (e<478.) and (e>=85.):
                se1fac = 0.25
    if (e<85.):
        se0fac = 0.091
    cssum0h2 = cssum0h2 + (csefh2h2p[j] + csefh2hp[j])*se0fac
    cssum1h2 = cssum1h2 + (csefh2h2p[j] + csefh2hp[j])*se1fac
    
#add in secondaries for sum0 to energy bin near 13 ev and for sum1 for energy bin near 36 eV
    
#secondary electrons that map to 13 eV
e = enh2[0]
ef = eflxforcs(e,xpass,ypass,Neval,Teval)
#store primary electron ionization p;roduction in lowest energy bin
pelh2h2p13 = csefh2hp[0]
pelh2hp13 = csefh2hp[0]
#store secondary electron ionization production in lowest energy bin
seplh2h2p13 = ef*csh2h2p[0]*cssum0h2
seplh2hp13 = ef*csh2hp[0]*cssum0h2
#store energy loss in lowest energy bin
elh2h2p13 = seplh2h2p13*13.
elh2hp13 = seplh2hp13*13.
#add secondary production with primary prouction for future calculation with altitude
irefh2h2p13 = seplh2h2p13/csefh2h2p[0]
csefh2h2p[0] = csefh2h2p[0] + seplh2h2p13
irefh2hp13 = seplh2hp13/csefh2hp[0]
csefh2hp[0] = csefh2hp[0] + seplh2hp13
#repeat for higher energy secondary electrons produced
e = enh2[4]
ef = eflxforcs(e,xpass,ypass,Neval,Teval)
seplh2h2p35 = ef*csh2h2p[4]*cssum1h2
seplh2hp35 = ef*csh2hp[4]*cssum1h2
elh2h2p35 = seplh2h2p35*35.
elh2hp35 = seplh2hp35*35.
irefh2h2p35 = seplh2h2p35/csefh2h2p[4]
csefh2h2p[4] = csefh2h2p[4] + seplh2h2p35
irefh2hp35 = seplh2hp35/csefh2hp[4]
csefh2hp[4] = csefh2hp[4] + seplh2hp35
#integrate over energy
pfh2h2p = integrate.simpson(csefh2h2p,enh2,even='avg')
pfh2hp = integrate.simpson(csefh2hp,enh2,even='avg')


#ionization of H2O
enh2o = np.array(dfCSH2O["E"],dtype=float)
csh2oh2op = np.array(dfCSH2O["H2O+"],dtype=float)
csh2oopp = np.array(dfCSH2O["O++"],dtype=float)
csh2oh2p = np.array(dfCSH2O["H2+"],dtype=float)
csh2ohp = np.array(dfCSH2O["H+"],dtype=float)
nepts = len(enh2o)
csefh2oh2op = np.zeros_like(csh2oh2op)
csefh2oopp = np.zeros_like(csh2oopp)
csefh2oh2p = np.zeros_like(csh2oh2p)
csefh2ohp = np.zeros_like(csh2ohp)
se0fac = 0.
cssum0h2o = 0.
se1fac =0.
cssum1h2o = 0.
for j in range(nepts):
    e = enh2o[j]
    ef = eflxforcs(e,xpass,ypass,Neval,Teval)
    csefh2oh2op[j] = ef*csh2oh2op[j]
    csefh2oopp[j] = ef*csh2oopp[j]
    csefh2oh2p[j] = ef*csh2oh2p[j]
    csefh2ohp[j] = ef*csh2ohp[j]
    
# include estimated effect of secondary electrons
    if (e>1746.):
        se1fac=0.45
    else:
        if (e<1746.) and (e>=478.):
            se1fac = 0.35
        else:
            if (e<478.) and (e>=85.):
                se1fac = 0.25
    if (e<85.):
        se0fac = 0.091
    cssum0h2o = cssum0h2o + (csefh2oh2op[j] + csefh2oopp[j] + csefh2oh2p[j] + csefh2ohp[j])*se0fac
    cssum1h2o = cssum1h2o + (csefh2oh2op[j] + csefh2oopp[j] + csefh2oh2p[j] + csefh2ohp[j])*se1fac
    
#add in secondaries for sum0 to energy bin near 13 ev and for sum1 for energy bin near 36 eV
    
#secondary electrons that map to 13 eV
e = enh2o[0]
ef = eflxforcs(e,xpass,ypass,Neval,Teval)
#store primary electron ionization production in lowest energy bin
pelh2oh2op13 = csefh2oh2op[0]
pelh2oopp13 = csefh2oopp[0]
pelh2oh2p13 = csefh2oh2p[0]
pelh2ohp13 = csefh2ohp[0]
#store secondary electron ionization production in lowest energy bin
seplh2oh2op13 = ef*csh2oh2op[0]*cssum0h2o
seplh2oopp13 = ef*csh2oopp[0]*cssum0h2o
seplh2oh2p13 = ef*csh2oh2p[0]*cssum0h2o
seplh2ohp13 = ef*csh2ohp[0]*cssum0h2o
#store energy loss in lowest energy bin
elh2oh2op13 = seplh2oh2op13*13.
elh2oopp13 = seplh2oopp13*13.
elh2oh2p13 = seplh2oh2p13*13.
elh2ohp13 = seplh2ohp13*13.
#add secondary production with primary prouction for future calculation with altitude
irefh2oh2op13 = seplh2oh2op13/csefh2oh2op[0]
csefh2oh2op[0] = csefh2oh2op[0] + seplh2oh2op13
irefh2oopp13 = seplh2oopp13/csefh2oopp[0]
csefh2oopp[0] = csefh2oopp[0] + seplh2oopp13
irefh2oh2p13 = seplh2oh2p13/csefh2oh2p[0]
csefh2oh2p[0] = csefh2oh2p[0] + seplh2oh2p13
irefh2ohp13 = seplh2ohp13/csefh2ohp[0]
csefh2ohp[0] = csefh2ohp[0] + seplh2ohp13
#repeat for higher energy secondary electrons produced
e = enh2o[7]
ef = eflxforcs(e,xpass,ypass,Neval,Teval)
seplh2oh2op35 = ef*csh2oh2op[7]*cssum1h2o
seplh2oopp35 = ef*csh2oopp[7]*cssum1h2o
seplh2oh2p35 = ef*csh2oh2p[7]*cssum1h2o
seplh2ohp35 = ef*csh2ohp[7]*cssum1h2o
elh2oh2op35 = seplh2oh2op35*35.
elh2oopp35 = seplh2oopp35*35.
elh2oh2p35 = seplh2oh2p35*35.
elh2ohp35 = seplh2ohp35*35.
irefh2oh2op35 = seplh2oh2op35/csefh2oh2op[7]
csefh2oh2op[7] = csefh2oh2op[7] + seplh2oh2op35
irefh2oopp35 = seplh2oopp35/csefh2oopp[7]
csefh2oopp[7] = csefh2oopp[7] + seplh2oopp35
irefh2oh2p35 = seplh2oh2p35/csefh2oh2p[7]
csefh2oh2p[7] = csefh2oh2p[7] + seplh2oh2p35
irefh2ohp35 = seplh2ohp35/csefh2ohp[7]
csefh2ohp[7] = csefh2ohp[7] + seplh2ohp35
#integrate over energy
pfh2oh2op = integrate.simpson(csefh2oh2op,enh2o,even='avg')
pfh2oopp = integrate.simpson(csefh2oopp,enh2o,even='avg')
pfh2oh2p = integrate.simpson(csefh2oh2p,enh2o,even='avg')
pfh2ohp = integrate.simpson(csefh2ohp,enh2o,even='avg')
                     
#dissociative excitation of O2
eno21304 = np.array(dfCSdeO21304["EO21304"],dtype=float)
cso21304 = np.array(dfCSdeO21304["CS1304Mb"],dtype=float)
nepts = len(eno21304)
csefo21304 = np.zeros_like(cso21304)
for j in range(nepts):
    e = eno21304[j]
    ef = eflxforcs(e,xpass,ypass,Neval,Teval)
    csefo21304[j] = ef*cso21304[j]
pfo21304 = integrate.simpson(csefo21304,eno21304,even='avg')
eno21356 = np.array(dfCSdeO21356["EO21356"],dtype=float)
cso21356 = np.array(dfCSdeO21356["CS1356Mb"],dtype=float)
nepts = len(eno21356)
csefo21356 = np.zeros_like(cso21356)
csefo2evtr1356 = np.zeros_like(cso21356)
for j in range(nepts):
    e = eno21356[j]
    ef = eflxforcs(e,xpass,ypass,Neval,Teval)
    csefo21356[j] = ef*cso21356[j]
    efevtr = eflxblw13(e,313.,1.160483225e6)
    if (e>2000.):
        efevtr = 0.0
    csefo2evtr1356[j] = efevtr*cso21356[j]
    
pfo21356 = integrate.simpson(csefo21356,eno21356,even='avg')
pfo21356evtr = integrate.simpson(csefo2evtr1356,eno21356,even='avg')

#dissociative excitation of H2O
enh2o1304 = np.array(dfCSdeH2O1304["EH2O1304"],dtype=float)
csh2o1304 = np.array(dfCSdeH2O1304["CS1304Mb$10"],dtype=float)
nepts = len(enh2o1304)
csefh2o1304 = np.zeros_like(csh2o1304)
for j in range(nepts):
    e = enh2o1304[j]
    ef = eflxforcs(e,xpass,ypass,Neval,Teval)
    csefh2o1304[j] = ef*csh2o1304[j]
pfh2o1304 = integrate.simpson(csefh2o1304,enh2o1304,even='avg')
enh2o1356 = np.array(dfCSdeH2O1356["EH2O1356"],dtype=float)
csh2o1356 = np.array(dfCSdeH2O1356["CS1356Mb$10"],dtype=float)
nepts = len(enh2o1356)
csefh2o1356 = np.zeros_like(csh2o1356)
csefh2oevtr1356 = np.zeros_like(csh2o1356)
for j in range(nepts):
    e = enh2o1356[j]
    ef = eflxforcs(e,xpass,ypass,Neval,Teval)
    csefh2o1356[j] = ef*csh2o1356[j]
    efevtr = eflxblw13(e,313,1.160483225e6)
    if (e>2000.):
        efevtr = 0.0
    csefh2oevtr1356[j] = efevtr*csh2o1356[j]
    
pfh2o1356 = integrate.simpson(csefh2o1356,enh2o1356,even='avg') 
pfh2o1356evtr = integrate.simpson(csefh2oevtr1356,enh2o1356,even='avg')

#calculate the column dissociation of O2 from the BtripletSigma cross section
eno2dis = np.array(dfCSB3sigo2["EO2B3sig"],dtype=float)
cso2dis = np.array(dfCSB3sigo2["CSB3sigeO2"],dtype=float)
nepts = len(eno2dis)
csefo2dis = np.zeros_like(cso2dis)
for j in range(nepts):
    e = eno2dis[j]
    ef = eflxforcs(e,xpass,ypass,Neval,Teval)
    csefo2dis[j] = ef*cso2dis[j]
pfo2dis = integrate.simpson(csefo2dis,eno2dis,even='avg')

#calculate the vibrational excitation of O2 and H2
eno2vib = np.array(dfCSvibo2["EO2vib"],dtype=float)
cso2vib = np.array(dfCSvibo2["CSvibeO2"],dtype=float)
nepts = len(eno2vib)
csefo2vib = np.zeros_like(cso2vib)
for j in range(nepts):
    e = eno2vib[j]
    ef = eflxforcs(e,xpass,ypass,Neval,Teval)
    csefo2vib[j] = ef*cso2vib[j]
pfo2vib = integrate.simpson(csefo2vib,eno2vib,even='avg')
enh2vib = np.array(dfCSvibh2["EH2vib"],dtype=float)
csh2vib = np.array(dfCSvibh2["CSvibeH2"],dtype=float)
nepts = len(enh2vib)
csefh2vib = np.zeros_like(csh2vib)
for j in range(nepts):
    e = enh2vib[j]
    ef = eflxforcs(e,xpass,ypass,Neval,Teval)
    csefh2vib[j] = ef*csh2vib[j]
pfh2vib = integrate.simpson(csefh2vib,enh2vib,even='avg')
    
#calculate the total scattering from O2
eno2scat = np.array(dfCSQTo2["EO2QT"],dtype=float)
cso2scat = np.array(dfCSQTo2["CSQTeO2"],dtype=float)
nepts = len(eno2scat)
csefo2scat = np.zeros_like(cso2scat)
for j in range(nepts):
    e = eno2scat[j]
    ef = eflxforcs(e,xpass,ypass,Neval,Teval)
    csefo2scat[j] = ef*cso2scat[j]
pfo2scat = integrate.simpson(csefo2scat,eno2scat,even='avg')

#iterate over atmsophereic density levels
atmscfactor = [1.,5.,100.,150.,200.]
teatmfactor = [2.67e3,1.5e2,1.5e2,1.5e2,1.5e2]
tnatmfactor = [1.5e2,1.5e2,1.5e2,1.5e2,1.5e2]
atmidx = len(atmscfactor)
for idx in range(atmidx):
#Solve sperately for dayside(n) and nightside(m)
    altgrd = np.array(dfAtm['alt'])
    altgrd = np.array(altgrd*1.0e2)
    nh2oden = np.array(dfAtm['H2On'])
    nh2oden = np.array(nh2oden/1.0e+6)
    no2den = np.array(dfAtm['O2n'])
    no2den = np.array(no2den/1.0e+6*atmscfactor[idx])
    nh2den = np.array(dfAtm['H2n'])
    nh2den = np.array(nh2den/1.0e+6*10.0)

    mh2oden = np.array(dfAtm['H2Om'])
    mh2oden = np.array(mh2oden/1.0e+6)
    mo2den = np.array(dfAtm['O2m'])
    mo2den = np.array(mo2den/1.0e+6*atmscfactor[idx])
    mh2den = np.array(dfAtm['H2m'])
    mh2den = np.array(mh2den/1.0e+6*10.0)

# O2 noise fix
    for i in range(len(altgrd)):
        if no2den[i]<1.e-1:
            no2den[i]=1.e-1
        if mo2den[i]<1.e-1:
            mo2den[i]=1.e-1
        
    ntden = nh2oden+no2den+nh2den
    mtden = mh2oden+mo2den+mh2den
    nso2mfp = 5.20e-15*no2den*no2den/ntden
    nsh2omfp = 1.66e-15*nh2oden*nh2oden/ntden
    nsh2mfp = 2.68e-15*nh2den*nh2den/ntden
    no2h2omfp = 1./4.*(5.20e-15**0.5+1.66e-15**0.5)**2*no2den*nh2oden/ntden
    no2h2mfp = 1./4.*(5.20e-15**0.5+2.68e-15**0.5)**2*no2den*nh2den/ntden
    nh2oh2mfp = 1./4.*(1.66e-15**0.5+2.68e-15**0.5)**2*nh2oden*nh2den/ntden
    nmfpm = (1./(2**0.5*(nso2mfp+nsh2omfp+nsh2mfp+no2h2omfp+no2h2mfp+nh2oh2mfp)))
    mso2mfp = 5.20e-15*mo2den*mo2den/mtden
    msh2omfp = 1.66e-15*mh2oden*mh2oden/mtden
    msh2mfp = 2.68e-15*mh2den*mh2den/mtden
    mo2h2omfp = 1./4.*(5.20e-15**0.5+1.66e-15**0.5)**2*mo2den*mh2oden/mtden
    mo2h2mfp = 1./4.*(5.20e-15**0.5+2.68e-15**0.5)**2*mo2den*mh2den/mtden
    mh2oh2mfp = 1./4.*(1.66e-15**0.5+2.68e-15**0.5)**2*mh2oden*mh2den/mtden
    mmfpm = (1./(2**0.5*(mso2mfp+msh2omfp+msh2mfp+mo2h2omfp+mo2h2mfp+mh2oh2mfp)))
    dfMFP = pd.DataFrame(data={'ALTITUDE': altgrd, 'noon mean free path m': nmfpm, 'midnight mean free path m': mmfpm})
    #Plot mean free path in km versus altitude in kilometers                     
    figmfp = px.scatter(x=nmfpm/1.0e5,y=altgrd/1.0e5,log_x=1,title="Mean Free Path (km)",labels={"y":  "Altitude(km)", "x": "Mean Free Path(km)"})
    figmfp.add_scatter(x=nmfpm/1.0e5,y=altgrd/1.0e5,name= 'Noon')
    figmfp.add_scatter(x=mmfpm/1.0e5,y=altgrd/1.0e5,name= 'Midnight')
    figmfp.update_xaxes(exponentformat='E', title='Mean Free Path (km)')
    figmfp.show()
#Plot for displaying the atmospheric density vs altitude
#noon
    figatmn = px.scatter(x=no2den, y=altgrd/1.0e5, log_x=1, title='Atmospheric densities (cm^-3)',labels={"y":  "Altitude(km)", "x": "Density(cm^-3)"})
    figatmn.add_scatter(x=no2den, y=altgrd/1.0e5, mode='markers',name='Day O2')
    figatmn.add_scatter(x=nh2oden, y=altgrd/1.0e5, mode='markers', name='Day H2O')
    figatmn.add_scatter(x=nh2den, y=altgrd/1.0e5,  mode='markers',name='Day H2')
    figatmn.update_xaxes(exponentformat="E")
    figatmn.show()
# Determine atmospheric column densities on dayside
    colnh2oden = integrate.simpson(nh2oden,altgrd,even='avg')
    colno2den = integrate.simpson(no2den,altgrd,even='avg')
    colnh2den = integrate.simpson(nh2den,altgrd,even='avg')
    print("NOON Column (cm^-2)", "H2O =", "{:.2e}".format(colnh2oden), "O2", "{:.2e}".format(colno2den),"H2", "{:.2e}".format(colnh2den))
#midnight
    figatmm = px.scatter(x=mo2den, y=altgrd/1.0e5, log_x=1, title='Atmospheric densities (cm^-3)',labels={"y":  "Altitude(km)", "x": "Density(cm^-3)"})
    figatmm.add_scatter(x=mo2den, y=altgrd/1.0e5, mode='markers',name='Night O2')
    figatmm.add_scatter(x=mh2oden, y=altgrd/1.0e5, mode='markers', name='Night H2O')
    figatmm.add_scatter(x=mh2den, y=altgrd/1.0e5,  mode='markers',name='Night H2')
    figatmm.update_xaxes(exponentformat="E")
    figatmm.show()
# Deternmmine column densities on the nightside
    colmh2oden = integrate.simpson(mh2oden,altgrd,even='avg')
    colmo2den = integrate.simpson(mo2den,altgrd,even='avg')
    colmh2den = integrate.simpson(mh2den,altgrd,even='avg')
    print("MIDNIGHT Column (cm^-2)", "H2O =", "{:.2e}".format(colmh2oden), "O2", "{:.2e}".format(colmo2den),"H2", "{:.2e}".format(colmh2den))

#Iterate over altitude
#for O2 processes
    nadpo2o2p = np.array(no2den*pfo2o2p)
    madpo2o2p = np.array(mo2den*pfo2o2p)
    nadpo2op = np.array(no2den*pfo2op)
    madpo2op = np.array(mo2den*pfo2op)
    nadpo2opp = np.array(no2den*pfo2opp)
    madpo2opp = np.array(mo2den*pfo2opp)
    nadpo21304 = np.array(no2den*pfo21304)
    madpo21304 = np.array(mo2den*pfo21304)
    nadpo21356 = np.array(no2den*pfo21356)
    nevtradpo21356 = np.array(no2den*pfo21356evtr)
    madpo21356 = np.array(mo2den*pfo21356)
    mevtradpo21356 = np.array(mo2den*pfo21356evtr)
    nadpo2dis = np.array(no2den*pfo2dis)
    madpo2dis = np.array(mo2den*pfo2dis)
    nadpo2vib = np.array(no2den*pfo2vib)
    madpo2vib = np.array(mo2den*pfo2vib)
    nadpo2scat = np.array(no2den*pfo2scat)
    madpo2scat = np.array(mo2den*pfo2scat)
#secondary electron production from O2 at 0=13eV and 1=35eV
    nadpse0o2 = np.array(cssum0o2*no2den)
    madpse0o2 = np.array(cssum0o2*mo2den)
    napdse1o2 = np.array(cssum1o2*no2den)
    mapdse1o2 = np.array(cssum1o2*mo2den)
#Generate DataStructure for processes as a function of altitude
    ProdO2Ndf_iteration = pd.DataFrame(data={'ATM':atmscfactor[idx],'ALT':altgrd,'NADPO2O2P':nadpo2o2p,'NADPO2OP':nadpo2op,'NADPO2OPP':nadpo2opp,'NADPO21304':nadpo21304,'NADPO21356':nadpo21356,'NADPO2dis':nadpo2dis,'NADPO2vib':nadpo2vib,'NADPO2scat':nadpo2scat})
    ProdO2Mdf_iteration = pd.DataFrame(data={'ATM':atmscfactor[idx],'ALT':altgrd,'MADPO2O2P':madpo2o2p,'MADPO2OP':madpo2op,'MADPO2OPP':madpo2opp,'MAPDO21304':madpo21304,'MADPO21356':madpo21356,'MADPO2dis':madpo2dis,'MADPO2vib':madpo2vib,'MADPO2scat':madpo2scat})
    ProdO2Ndf = ProdO2Ndf.append(ProdO2Ndf_iteration,ignore_index = True)
    ProdO2Mdf = ProdO2Mdf.append(ProdO2Mdf_iteration,ignore_index = True)
    
#for H2O processes
    nadph2oh2op = np.array(nh2oden*pfh2oh2op)
    madph2oh2op = np.array(mh2oden*pfh2oh2op)
    nadph2oopp = np.array(nh2oden*pfh2oopp)
    madph2oopp = np.array(mh2oden*pfh2oopp)
    nadph2oh2p = np.array(nh2oden*pfh2oh2p)
    madph2oh2p = np.array(mh2oden*pfh2oh2p)
    nadph2ohp = np.array(nh2oden*pfh2ohp)
    madph2ohp = np.array(mh2oden*pfh2ohp)
    nadph2o1304 = np.array(nh2oden*pfh2o1304)
    madph2o1304 = np.array(mh2oden*pfh2o1304)
    nadph2o1356 = np.array(nh2oden*pfh2o1356)
    nevtradph2o1356 = np.array(nh2oden*pfh2o1356evtr)
    madph2o1356 = np.array(mh2oden*pfh2o1356)
    mevtradph2o1356 = np.array(mh2oden*pfh2o1356evtr)
#secondary electron production from H2O at 0=13eV and 1=35eV
    nadpse0h2o = np.array(cssum0h2o*nh2oden)
    madpse0h2o = np.array(cssum0h2o*mh2oden)
    nadpse1h2o = np.array(cssum1h2o*nh2oden)
    madpse1h2o = np.array(cssum1h2o*mh2oden)
#Generate DataStructure for processes as a function of altitude
    ProdH2ONdf_iteration = pd.DataFrame(data={'ATM':atmscfactor[idx],'ALT':altgrd,'NADPH2OH2OP':nadph2oh2op,'NADPH2OOPP':nadph2oopp,'NADPH2OH2P':nadph2oh2p,'NADPH2OHP':nadph2ohp,'NAPDH2O1304':nadph2o1304,'NAPDH2O1356':nadph2o1356})
    ProdH2OMdf_iteration = pd.DataFrame(data={'ATM':atmscfactor[idx],'ALT':altgrd,'MADPH2OH2OP':madph2oh2op,'MADPH2OOPP':madph2oopp,'MADPH2OH2P':madph2oh2p,'MADPH2OHP':madph2ohp,'MAPDH2O1304':madph2o1304,'MAPDH2O1356':madph2o1356})
    ProdH2ONdf = ProdH2ONdf.append(ProdH2ONdf_iteration,ignore_index = True)
    ProdH2OMdf = ProdH2OMdf.append(ProdH2OMdf_iteration,ignore_index = True)    

#Generate DataStructure for processes as a function of altitude
    ProdH2ONdf_iteration = pd.DataFrame(data={'ATM':atmscfactor[idx],'ALT':altgrd,'NADPH2OH2OP':nadph2oh2op,'NADPH2OOPP':nadph2oopp,'NADPH2OH2P':nadph2oh2p,'NADPH2OHP':nadph2ohp,'NAPDH2O1304':nadph2o1304,'NAPDH2O1356':nadph2o1356})
    ProdH2OMdf_iteration = pd.DataFrame(data={'ATM':atmscfactor[idx],'ALT':altgrd,'MADPH2OH2OP':madph2oh2op,'MADPH2OOPP':madph2oopp,'MADPH2OH2P':madph2oh2p,'MADPH2OHP':madph2ohp,'MAPDH2O1304':madph2o1304,'MAPDH2O1356':madph2o1356})
    ProdH2ONdf = ProdH2ONdf.append(ProdH2ONdf_iteration,ignore_index = True)
    ProdH2OMdf = ProdH2OMdf.append(ProdH2OMdf_iteration,ignore_index = True)

#for H2 processes
    nadph2h2p = np.array(nh2den*pfh2h2p)
    madph2h2p = np.array(mh2den*pfh2h2p)
    nadph2hp = np.array(nh2den*pfh2hp)
    madph2hp = np.array(mh2den*pfh2hp)
    nadph2vib = np.array(nh2den*pfh2vib)
    madph2vib = np.array(mh2den*pfh2vib)
#secondary electron production from H2 at 0=13eV and 1=35eV
    nadpse0h2 = np.array(cssum0h2*nh2den)
    madpse0h2 = np.array(cssum0h2*mh2den)
    napdse1h2 = np.array(cssum1h2*nh2den)
    mapdse1h2 = np.array(cssum1h2*mh2den)
#Generate DataStructure for processes as a function of altitude
    ProdH2Ndf_iteration = pd.DataFrame(data={'ATM':atmscfactor[idx],'ALT':altgrd,'NADPH2H2P':nadph2h2p,'NADPH2HP':nadph2hp,'NADPH2vib':nadph2vib})
    ProdH2Mdf_iteration = pd.DataFrame(data={'ATM':atmscfactor[idx],'ALT':altgrd,'MADPH2H2P':madph2h2p,'MADPH2HP':madph2hp,'MADPH2vib':madph2vib})
    ProdH2Ndf = ProdH2Ndf.append(ProdH2Ndf_iteration,ignore_index = True)
    ProdH2Mdf = ProdH2Mdf.append(ProdH2Mdf_iteration,ignore_index = True)
#Calculate secondary electron production of all processes but ionization, which is already included in CS
    nadpseh2vib = np.array((nadpse0h2+nadpse0h2o+nadpse0o2)*csh2vib[4]*nh2den)
    nadpseo2vib = np.array(((nadpse0h2+nadpse0h2o+nadpse0o2)*cso2vib[5])+((nadpse0h2+nadpse0h2o+nadpse0o2)*cso2vib[7])*no2den)
    nadpseo2dis = np.array(((nadpse0h2+nadpse0h2o+nadpse0o2)*cso2dis[0])+((nadpse0h2+nadpse0h2o+nadpse0o2)*cso2dis[5])*no2den)
    nadpseo21304 = np.array(((nadpse0h2+nadpse0h2o+nadpse0o2)*cso21304[0])+((nadpse0h2+nadpse0h2o+nadpse0o2)*cso21304[6])*no2den)
    nadpseo21356 = np.array(((nadpse0h2+nadpse0h2o+nadpse0o2)*cso21356[0])+((nadpse0h2+nadpse0h2o+nadpse0o2)*cso21356[6])*no2den)
    nadpseh2o1304 = np.array(((nadpse0h2+nadpse0h2o+nadpse0o2)*cso21304[0])+((nadpse0h2+nadpse0h2o+nadpse0o2)*cso21304[8])*nh2oden)
    nadpseh2o1356 = np.array(((nadpse0h2+nadpse0h2o+nadpse0o2)*cso21356[0])+((nadpse0h2+nadpse0h2o+nadpse0o2)*cso21356[8])*nh2oden)

#Generate emission rate profiles for each atmospheric case to compare with Tommy Greathouse
    OI1356 = np.array(nadph2o1356+nadpo21356)
    OI1304 = np.array(nadph2o1304+nadpo21304)
    ProdERPdf_iteration = pd.DataFrame(data={'ATM':atmscfactor[idx],'ALT':altgrd,'OI1356':OI1356,'OI1304':OI1304})
    ProdERPdf = ProdERPdf.append(ProdERPdf_iteration,ignore_index = True)

#Calculate altitude dependent energy loss
#For O2
    nadpelo2pt1 = np.array((nadpo2vib+nadpseo2vib)*0.23+(nadpo2dis+nadpseo2dis)*5+(nadpo21304+nadpseo21304+nadpo21356+nadpseo21356)*14.8)
    nadpelo2pt2 = np.array((nadpo2o2p*13.)+(nadpo2op*23.)+(nadpo2opp*73.))
    nadpelo2=np.array(nadpelo2pt1+nadpelo2pt2)
#For H2
    nadpelh2 = np.array((nadph2vib+nadpseh2vib)*0.5159 + (nadph2h2p*17.) + (nadph2hp*25.))
#For H2O
    nadpelh2opt1 = np.array((nadph2o1304+nadph2o1356+nadpseh2o1304+nadpseh2o1356)*16.)
    nadpelh2opt2 = np.array((nadph2oh2op*13.)+(nadph2oopp*90.)+(nadph2oh2p*30.)+(nadph2ohp*20.))
    nadpelh2o=np.array(nadpelh2opt1+nadpelh2opt2)
#Total energy loss as a function of altitude in units of eV m^-3
    nadpelT = np.array(nadpelo2+nadpelh2+nadpelh2o)
    
#integrate all processes over the full altitude range and put integrated emissions in Rayleighs
                
#ionization
    ncpo2o2p = integrate.simpson((nadpo2o2p+photo2o2p),altgrd,even='avg')
    mcpo2o2p = integrate.simpson(madpo2o2p,altgrd,even='avg')
    ncpo2op = integrate.simpson((nadpo2op+photo2op),altgrd,even='avg')
    mcpo2op = integrate.simpson(madpo2op,altgrd,even='avg')
    ncpo2opp = integrate.simpson(nadpo2opp,altgrd,even='avg')
    mcpo2opp = integrate.simpson(madpo2opp,altgrd,even='avg')
    ncph2oh2op = integrate.simpson((nadph2oh2op+photh2oh2op),altgrd,even='avg')
    mcph2oh2op = integrate.simpson(madph2oh2op,altgrd,even='avg')
    ncph2oopp = integrate.simpson(nadph2oopp,altgrd,even='avg')
    mcph2oopp = integrate.simpson(madph2oopp,altgrd,even='avg')
    ncph2oh2p = integrate.simpson(nadph2oh2p,altgrd,even='avg')
    mcph2oh2p = integrate.simpson(madph2oh2p,altgrd,even='avg')
    ncph2ohp = integrate.simpson((nadph2ohp+photh2ohp),altgrd,even='avg')
    mcph2ohp = integrate.simpson(madph2ohp,altgrd,even='avg')
    ncph2h2p = integrate.simpson((nadph2h2p+photh2h2p),altgrd,even='avg')
    mcph2h2p = integrate.simpson(madph2h2p,altgrd,even='avg')
    ncph2hp = integrate.simpson((nadph2hp+photh2hp),altgrd,even='avg')
    mcph2hp = integrate.simpson(madph2hp,altgrd,even='avg')
    
#Dissociation of O2 only for now
    ncpo2dis = integrate.simpson(nadpo2dis,altgrd,even='avg')
    mcpo2dis = integrate.simpson(madpo2dis,altgrd,even='avg')
#for emission of 1304 and 1356
    nRo21304 = (integrate.simpson(nadpo21304,altgrd,even='avg'))/1.0e+6
    mRo21304 = (integrate.simpson(madpo21304,altgrd,even='avg'))/1.0e+6
    nRo21356 = (integrate.simpson(nadpo21356,altgrd,even='avg'))/1.0e+6
    mRo21356 = (integrate.simpson(madpo21356,altgrd,even='avg'))/1.0e+6
    nRh2o1304 = (integrate.simpson(nadph2o1304,altgrd,even='avg'))/1.0e+6
    mRh2o1304 = (integrate.simpson(madph2o1304,altgrd,even='avg'))/1.0e+6
    nRh2o1356 = (integrate.simpson(nadph2o1356,altgrd,even='avg'))/1.0e+6
    mRh2o1356 = (integrate.simpson(madph2o1356,altgrd,even='avg'))/1.0e+6
    nevtro21356 = (integrate.simpson(nevtradpo21356,altgrd,even='avg'))/1.0e+6
    nevtrh2o1356 = (integrate.simpson(nevtradph2o1356,altgrd,even='avg'))/1.0e+6
    mevtro21356 = (integrate.simpson(mevtradpo21356,altgrd,even='avg'))/1.0e+6
    mevtrh2o1356 = (integrate.simpson(mevtradph2o1356,altgrd,even='avg'))/1.0e+6
#Generate a data structure for the altitude inegrated production for each ATM
    CProdH2ONdf_iteration = pd.DataFrame(data={'ATM':atmscfactor[idx],'NCPH2OH2OP':ncph2oh2op,'NCPH2OOPP':ncph2oopp,'NCPH2OH2P':ncph2oh2p,'NCPH2OHP':ncph2ohp,'NRH2O1304':nRh2o1304,'NRH2O1356':nRh2o1356},index=[0])
    CProdH2OMdf_iteration = pd.DataFrame(data={'ATM':atmscfactor[idx],'MCPH2OH2OP':mcph2oh2op,'MCPH2OOPP':mcph2oopp,'MCPH2OH2P':mcph2oh2p,'MCPH2OHP':mcph2ohp,'MRH2O1304':mRh2o1304,'MRH2O1356':mRh2o1356},index=[0])
    CProdH2ONdf = CProdH2ONdf.append(CProdH2ONdf_iteration,ignore_index = True)
    CProdH2OMdf = CProdH2OMdf.append(CProdH2OMdf_iteration,ignore_index = True)
    CProdO2Ndf_iteration = pd.DataFrame(data={'ATM':atmscfactor[idx],'NCPO2O2P':ncpo2o2p,'NCPO2OP':ncpo2op,'NCPO2OPP':ncpo2opp,'NRO21304':nRo21304,'NRO21356':nRo21356,'NCPO2dis':ncpo2dis},index=[0])
    CProdO2Mdf_iteration = pd.DataFrame(data={'ATM':atmscfactor[idx],'MCPO2O2P':mcpo2o2p,'MCPO2OP':mcpo2op,'MCPO2OPP':mcpo2opp,'MRO21304':mRo21304,'MRO21356':mRo21356,'MCPO2dis':mcpo2dis},index=[0])
    CProdO2Ndf = CProdO2Ndf.append(CProdO2Ndf_iteration,ignore_index = True)
    CProdO2Mdf = CProdO2Mdf.append(CProdO2Mdf_iteration,ignore_index = True)
    CProdH2Ndf_iteration = pd.DataFrame(data={'ATM':atmscfactor[idx],'NCPH2H2P':ncph2h2p,'NCPH2HP':ncph2hp},index=[0])
    CProdH2Mdf_iteration = pd.DataFrame(data={'ATM':atmscfactor[idx],'MCPH2H2P':mcph2h2p,'MCPH2HP':mcph2hp},index=[0])
    CProdH2Ndf = CProdH2Ndf.append(CProdH2Ndf_iteration,ignore_index = True)
    CProddfH2M = CProdH2Mdf.append(CProdH2Mdf_iteration,ignore_index = True)
    
# Electron energy portion of the code:
# consider only energetic electron energy input to thermal electrons for heating
# consider only H2 and O2 vibrational cooling of thermal electrons 
# iterate over altitude and energetic electron energy

# Extend the energy range of the incoming electrons to include 
# the Maxwellian electrons at 25,000 K that match 
# the Waves measured electrons from the plasma disk
    idxt=5+len(dfenergy)
    energyplus = np.zeros((idxt,),dtype=float)
    energyfluxplus = np.zeros((idxt,),dtype=float)
    energyhotpdisk = [0.5,1.,2.,4.,8.,12.]
    energyfluxhotpdisk = np.zeros_like(energyhotpdisk)
    energyfluxhotpdisk[0] = eflxblw13(0.5,1.0,2.4e4)
    energyfluxhotpdisk[1] = eflxblw13(1.0,1.0,2.4e4)
    energyfluxhotpdisk[2] = eflxblw13(2.0,1.0,2.4e4)
    energyfluxhotpdisk[3] = eflxblw13(4.0,1.0,2.4e4)
    energyfluxhotpdisk[4] = eflxblw13(8.0,1.0,2.4e4)
    energyfluxhotpdisk[5] = eflxblw13(12.0,1.0,2.4e4)
    energyplus[0] = energyhotpdisk[0]
    energyplus[1] = energyhotpdisk[1]
    energyplus[2] = energyhotpdisk[2]
    energyplus[3] = energyhotpdisk[3]
    energyplus[4] = energyhotpdisk[4]
    energyplus[5] = energyhotpdisk[5]
    energyfluxplus[0] = energyfluxhotpdisk[0]
    energyfluxplus[1] = energyfluxhotpdisk[1]
    energyfluxplus[2] = energyfluxhotpdisk[2]
    energyfluxplus[3] = energyfluxhotpdisk[3]
    energyfluxplus[4] = energyfluxhotpdisk[4]
    energyfluxplus[5] = energyfluxhotpdisk[5]
    idxplus = len(energyplus)-1
    for k in range(6,idxplus):
        energyplus[k] = dfenergy[k-6]
        energyfluxplus[k] =dfelecflux[k-6]
    Te=teatmfactor[idx]
    Tn=1.5e2  
    hrt = np.zeros_like(altgrd)
    hrtperener = np.zeros_like(energyplus)
    crt = np.zeros_like(altgrd)
# generate an electron altitude profile ingress eflg=0, egress eflg=1
    elecalt = elecaltprofile(0,altgrd)
    nalt=len(altgrd)-1
    nealtarray = np.zeros_like(altgrd)
    for i in range(nalt):
        nealtarray[i] = (elecalt[i]+elecalt[i+1])/2.
# define the energetic electron transfer rate in eV cm^-3 s^-1 using Swartz et al., JGR, &6, 8425, 1971.
    def ETtrans(ne,Te,E,eflx):
        Ee=8.6171e-5*Te
        Eterm = ((E-Ee)/(E-0.53*Ee))
        Etermp = pow(Eterm,2.36)
        Ep = pow(E,0.94)
        nep = pow(ne,0.97)
# Notice this is ne*L(E,e)
        ET = ((3.37e-12*nep)/Ep)*Etermp
        EFT = ET*eflx
        return EFT
# iterate over the energy levels of the energetic electrons and build up an altitude
# dependent electron heating rate array
    def elechrt(altgrd,energy,elecflux,nealtarray,Te):
        nalt=len(altgrd)-1
        jener = len(energyplus)-1
        for i in range(nalt):
            nealt = nealtarray[i]
            for j in range(jener):
                hrtperener[j] = ETtrans(nealt,Te,energyplus[j],energyfluxplus[j])
            hrt[i] = integrate.simpson(hrtperener,energyplus,even='avg')
        return hrt

# Cooling of thermal electrons comes from H2 and O2 rotational and vibrational excitation
# Use Schunk and Nagy, 'Ionosphere reference'
    def eleccrt(altgrd,elecalt,Tn,Te,o2den,h2den):
        k = 8.6171e-5
        Tchk = Te-Tn
        Lo2r = np.array(6.9e-14*elecalt*o2den*(Tchk/np.sqrt(Te)))
        h=(3300.-(839.*np.sin(1.91e-4*(Te-2700.))))
        Lo2v = (-1.)*np.array(5.2e-13*elecalt*o2den*np.exp(h*(Te-700.)/(700.*Te))*(np.exp((-1.)*(2770.*((Tchk)/(Te*Tn))))-1.))
        Lh2r = np.array(2.278e-11*elecalt*h2den*(np.exp(2.093e-4*(pow(Tchk,1.078)-1.))))
        w=0.54
        p1=pow(Te,0.5)
        p2=pow(Te,1.5)
        p3=pow(Te,2.0)
        I1=np.exp((8.554e4/p3)-34.95-(434./p1)-(3.915e4/p2))
        I2=np.exp((3.573e5/p3)-34.85-(673.3/p1)-(1.066e5/p2))
#        print(I1,I2)
        Lh2v = (8.37e13*elecalt*h2den*w/p2)*(I1*(1.-np.exp((w/k)*((1./Te)-(1./Tn))))+2.*I2*(1.-np.exp((2.*w/k)*((1./Te)-(1./Tn)))))
#        print(altgrd,Lo2r,Lo2v,Lh2r,Lh2v)
        crt = np.array(Lo2r+Lh2r+Lo2v+Lh2v)
        return crt
# Calculate integrated heating and cooling rates
    ihrt = integrate.simpson(elechrt(altgrd,energyplus,energyfluxplus,nealtarray,Te),altgrd,even='avg')
    icrt = integrate.simpson(eleccrt(altgrd,elecalt,Tn,Te,no2den,nh2den),altgrd,even='avg')
    nethcrt = ihrt-icrt
    
# plot heating and colling rates versus altitude
    figenergy = px.scatter(x=elechrt(altgrd,energyplus,energyfluxplus,nealtarray,Te), y=altgrd/1.0e5, log_x=1, title='Electron Heating and Cooling Rates ',labels={"y":  "Altitude(km)", "x": "Heating Rate(eV cm^-3 s^-1)"})
    figenergy.add_scatter(x=elechrt(altgrd,energyplus,energyfluxplus,nealtarray,Te), y=altgrd/1.0e5, mode='markers',name='HRT')
    figenergy.add_scatter(x=eleccrt(altgrd,elecalt,Tn,Te,no2den,nh2den), y=altgrd/1.0e5, mode='markers', name='CRT')
    figenergy.update_xaxes(exponentformat="E")
    figenergy.show()
    print('net integrated heating-cooling rates',"{:.2e}".format(nethcrt))
             
    
#Form useful column production rate sums for noon
#O2+ by O2 and H2O
    ncpo2p = ncpo2o2p    
    print("NOON","O2+ =", "{:.2e}".format(ncpo2p))
#H2+ by H2O and H2
    ncph2p = ncph2oh2p + ncph2h2p 
    nrh2h2o = ncph2h2p/ncph2oh2p
    print("NOON", "H2+ =", "{:.2e}".format(ncph2p), "PH2/PH2O =", "{:.2e}".format(nrh2h2o))
#mass 16 - water group - from H2O and O2
    ncpwg = ncph2oh2op + ncpo2op
    nrwgo2 = ncph2oh2op/ncpo2op
    print("NOON","WG+=", "{:.2e}".format(ncpwg), "WG/O2 =", "{:.2e}".format(nrwgo2))
#H+ column
    ncphp = ncph2hp + ncph2ohp
    print("NOON H+=", "{:.2e}".format(ncphp)) 
#emission totals
    NR1304 = nRo21304 + nRh2o1304
    nro2h2o1304 = nRo21304/nRh2o1304
    NR1356 = nRo21356 + nRh2o1356
    nro2h2o1356 = nRo21356/nRh2o1356
    NR13561304 = NR1356/NR1304
    print(atmscfactor[idx])
    print('NOON','1304 in Rayleighs = ',"{:.2e}".format(NR1304), 'PO2/PH2O = ', "{:.2e}".format(nro2h2o1304))
    print('NOON','1356 in Rayleighs = ',"{:.2e}".format(NR1356), 'PO2/PH2O = ', "{:.2e}".format(nro2h2o1356))
    print('NOON','1356/1304 ratio = ', "{:.2e}".format(NR13561304))
# Eviatar emission check
    print('noon eviatar o21356 in Rayleighs', "{:.2e}".format(nevtro21356))
    print('noon eviatar h2o1356 in Rayleighs', "{:.2e}".format(nevtrh2o1356))
#Form useful column production rate sums for midnight
#O2+ by O2 and H2O
    mcpo2p = mcpo2o2p 
    print("MIDNIGHT","O2+ =", "{:.2e}".format(ncpo2p))
#H2+ by H2O and H2
    mcph2p = mcph2oh2p + mcph2h2p
    mrh2h2o = mcph2h2p/mcph2oh2p
    print("MIDNIGHT", "H2+ =", "{:.2e}".format(ncph2p), "PH2/PH2O =", "{:.2e}".format(nrh2h2o))
#mass 16 - water group - from H2O and O2
    mcpwg = mcph2oh2op + mcpo2op
    mrwgo2 = mcph2oh2op/mcpo2op
    print("MIDNIGHT","WG+=", "{:.2e}".format(ncpwg), "WG/O2 =", "{:.2e}".format(nrwgo2))
#  H+
    mcphp = mcph2hp + mcph2ohp
    print("MIDNIGHT H+=", "{:.2e}".format(mcphp)) 
#emission totals
    MR1304 = mRo21304 + mRh2o1304
    mro2h2o1304 = mRo21304/mRh2o1304
    MR1356 = mRo21356 + mRh2o1356
    mro2h2o1356 = mRo21356/mRh2o1356
    MR13561304 = MR1356/MR1304
    print('MIDNIGHT','1304 in Rayleighs = ',"{:.2e}".format(MR1304), 'PO2/PH2O = ', "{:.2e}".format(mro2h2o1304))
    print('MIDNIGHT','1356 in Rayleighs = ',"{:.2e}".format(MR1356), 'PO2/PH2O = ', "{:.2e}".format(mro2h2o1356))
    print('MIDNIGHT','1356/1304 ratio = ', "{:.2e}".format(MR13561304))
# Eviatar emission check
    print('midnight eviatar o21356 in Rayleighs', "{:.2e}".format(mevtro21356))
    print('midnight eviatar h2o1356 in Rayleighs', "{:.2e}".format(mevtrh2o1356))
#    print('ionization at midnight',mcpo2o2p,mcpo2op,mcpo2opp,mcph2oh2op,mcph2oo2p,mcph2oh2p,mcph2ohp,mcph2h2p,mcph2hp)
#    print('ionization at noon',ncpo2o2p,ncpo2op,ncpo2opp,ncph2oh2op,ncph2oo2p,ncph2oh2p,ncph2ohp,ncph2h2p,ncph2hp)
#    print('emission at midnight',mRo21304,mRo21356,mRh2o1304,mRh2o1356)
#    print('emission at noon',nRo21304,nRo21356,nRh2o1304,nRh2o1356)
#compare sum of incoming electrons to total ionization production
    elecsum = 0.
    ypassm2 = np.array(ypass)
    elecsum = (integrate.simpson(ypassm2,xpass,even='avg'))
    ynew = np.array(xpass*ypassm2)
    energysum = (integrate.simpson(ynew,xpass,even='avg'))
#    print(xpass,ypassm2,ynew)
#sum of ions produced
    nsumofions = ncpo2p + ncph2p + ncpwg + ncph2ohp + ncph2hp + ncpo2opp
    msumofions = mcpo2p + mcph2p + mcpwg + mcph2ohp + mcph2hp + mcpo2opp
    nenergyions = (25.+12.3)*nsumofions
    menergyions = (25.+12.3)*msumofions
# produced ions to incoming electrons
    nripein = nsumofions/elecsum
    mripein = msumofions/elecsum
    nenergyio = energysum/nenergyions
    menergyio = energysum/menergyions
    Tenergyflx = energysum*1.6e-12
    print('nsumofions',"{:.2e}".format(nsumofions), 'msumofions',"{:.2e}".format(msumofions),'esum',"{:.2e}".format(elecsum))
    print( 'Noon: Prod Ions / Electron In',"{:.2e}".format(nripein), 'Midnight: Prod Ions / Electron In',"{:.2e}".format(mripein))
    print( 'Noon: Energy in / Energy out',"{:.2e}".format(nenergyio), 'Midnight: Energy in / Energy Out',"{:.2e}".format(menergyio))
    print('Total Energy input in ergs per cm squared',"{:.2e}".format(Tenergyflx))
#Energy loss calculation
    Tenergyflx = energysum*1.6e-12
#Calculate energy loss in each atmospheric slab in mW/m^2 for noon profile
    nelsum = 0.0
    nelperslab = np.zeros_like(altgrd)
    for i in range (len(altgrd)-2):
        altinc = altgrd[i+1] - altgrd[i]
        nelperslab[i] = altinc*nadpelT[i]*1.6e-12
        nelsum = nelsum + nelperslab[i]
    nelperslab[(len(altgrd)-1)] = nelperslab[(len(altgrd)-2)]
    nelet = nelsum/Tenergyflx
#Plot energy loss in ergs cm^-3 versus altitude in kilometers
    dfEL = pd.DataFrame(data={'Altitude': altgrd, 'noon energy loss': nelperslab})
    figel = px.scatter(x=nelperslab,y=altgrd/1.0e5,log_x=1,title="NOON: Energy Loss (ergs cm^-3)",labels={"y":  "Altitude(km)", "x": "Energy Loss (ergs cm^-3)"})
    figel.update_xaxes(exponentformat='E', title='Energy Loss (ergs cm^-3)')
    figel.show()   
    
#Write the emission rates profile to csv
ProdERPdf.to_csv("Emission_Rate_Profiles_Ganymede.csv")


NOON Column (cm^-2) H2O = 1.41e+16 O2 1.24e+14 H2 2.88e+13


MIDNIGHT Column (cm^-2) H2O = 1.04e+10 O2 2.21e+14 H2 7.86e+13


Photolysis producing H2+ 5.77e+04
Photolysis producing H+ 6.83e+06
Photolysis producing m16+ 2.05e+08
Photolysis producing O2+ 2.14e+06
called eflxblw13 2077693.3164193546
called eflxblw13 1098736.9715373197
called eflxblw13 572977.5185731861
called eflxblw13 7858498.624303063
called eflxblw13 7120872.934417763
called eflxblw13 5595600.0428923685
called eflxblw13 3095861.5592858307
called eflxblw13 1098736.9715373197
called eflxblw13 7165772.205886873
called eflxblw13 9009411.207171563
called eflxblw13 8495541.858687507
called eflxblw13 2224215.6603596723
called eflxblw13 1098736.9715373197
called eflxblw13 1088177.956783339
called eflxblw13 4221155.260492528
called eflxblw13 1098736.9715373197


<ipython-input-3-63c80e6c86bc>:407: RuntimeWarning:

invalid value encountered in double_scalars

<ipython-input-3-63c80e6c86bc>:409: RuntimeWarning:

invalid value encountered in double_scalars

<ipython-input-3-63c80e6c86bc>:424: RuntimeWarning:

invalid value encountered in double_scalars

<ipython-input-3-63c80e6c86bc>:482: RuntimeWarning:

invalid value encountered in double_scalars

<ipython-input-3-63c80e6c86bc>:560: RuntimeWarning:

invalid value encountered in double_scalars

<ipython-input-3-63c80e6c86bc>:562: RuntimeWarning:

invalid value encountered in double_scalars

<ipython-input-3-63c80e6c86bc>:564: RuntimeWarning:

invalid value encountered in double_scalars

<ipython-input-3-63c80e6c86bc>:579: RuntimeWarning:

invalid value encountered in double_scalars



NOON Column (cm^-2) H2O = 1.41e+16 O2 1.24e+14 H2 2.88e+14


...

MIDNIGHT Column (cm^-2) H2O = 1.04e+10 O2 3.31e+16 H2 7.86e+14


net integrated heating-cooling rates 2.77e+06
NOON O2+ = 3.21e+09
NOON H2+ = 2.12e+07 PH2/PH2O = 7.38e+00
NOON WG+= 4.44e+09 WG/O2 = 1.64e+00
NOON H+= 5.31e+08
150.0
NOON 1304 in Rayleighs =  4.31e+01 PO2/PH2O =  8.60e+00
NOON 1356 in Rayleighs =  8.47e+01 PO2/PH2O =  9.22e+02
NOON 1356/1304 ratio =  1.97e+00
noon eviatar o21356 in Rayleighs 4.70e+05
noon eviatar h2o1356 in Rayleighs 6.57e+02
MIDNIGHT O2+ = 3.21e+09
MIDNIGHT H2+ = 2.12e+07 PH2/PH2O = 7.38e+00
MIDNIGHT WG+= 4.44e+09 WG/O2 = 1.64e+00
MIDNIGHT H+= 3.66e+06
MIDNIGHT 1304 in Rayleighs =  6.88e+01 PO2/PH2O =  2.08e+07
MIDNIGHT 1356 in Rayleighs =  1.51e+02 PO2/PH2O =  2.23e+09
MIDNIGHT 1356/1304 ratio =  2.19e+00
midnight eviatar o21356 in Rayleighs 8.38e+05
midnight eviatar h2o1356 in Rayleighs 4.84e-04
nsumofions 8.24e+09 msumofions 8.84e+09 esum 4.65e+09
Noon: Prod Ions / Electron In 1.77e+00 Midnight: Prod Ions / Electron In 1.90e+00
Noon: Energy in / Energy out 1.68e+02 Midnight: Energy in / Energy Out 1.56e+02
Total En

NOON Column (cm^-2) H2O = 1.41e+16 O2 2.47e+16 H2 2.88e+14


MIDNIGHT Column (cm^-2) H2O = 1.04e+10 O2 4.41e+16 H2 7.86e+14


net integrated heating-cooling rates 2.77e+06
NOON O2+ = 4.29e+09
NOON H2+ = 2.12e+07 PH2/PH2O = 7.38e+00
NOON WG+= 5.00e+09 WG/O2 = 1.23e+00
NOON H+= 5.31e+08
200.0
NOON 1304 in Rayleighs =  5.59e+01 PO2/PH2O =  1.15e+01
NOON 1356 in Rayleighs =  1.13e+02 PO2/PH2O =  1.23e+03
NOON 1356/1304 ratio =  2.02e+00
noon eviatar o21356 in Rayleighs 6.26e+05
noon eviatar h2o1356 in Rayleighs 6.57e+02
MIDNIGHT O2+ = 4.29e+09
MIDNIGHT H2+ = 2.12e+07 PH2/PH2O = 7.38e+00
MIDNIGHT WG+= 5.00e+09 WG/O2 = 1.23e+00
MIDNIGHT H+= 3.66e+06
MIDNIGHT 1304 in Rayleighs =  9.17e+01 PO2/PH2O =  2.78e+07
MIDNIGHT 1356 in Rayleighs =  2.01e+02 PO2/PH2O =  2.98e+09
MIDNIGHT 1356/1304 ratio =  2.19e+00
midnight eviatar o21356 in Rayleighs 1.12e+06
midnight eviatar h2o1356 in Rayleighs 4.84e-04
nsumofions 9.88e+09 msumofions 1.18e+10 esum 4.65e+09
Noon: Prod Ions / Electron In 2.12e+00 Midnight: Prod Ions / Electron In 2.53e+00
Noon: Energy in / Energy out 1.40e+02 Midnight: Energy in / Energy Out 1.17e+02
Total En